In [1]:
import jinja2
import os, glob
from pprint import pformat
import numpy as np
import pandas as pd
import copy

In [7]:
colors = ((230, 25, 75), (60, 180, 75), (255, 225, 25), (0, 130, 200), 
          (245, 130, 48), (145, 30, 180), (70, 240, 240), (240, 50, 230),
          (210, 245, 60), (250, 190, 190), (0, 128, 128), (230, 190, 255),
          (170, 110, 40), (255, 250, 200), (128, 0, 0), (170, 255, 195), 
          (128, 128, 0), (255, 215, 180))

def render(tpl_path, context):
    path, filename = os.path.split(tpl_path)

    environment = jinja2.Environment(undefined=jinja2.StrictUndefined,
        loader=jinja2.FileSystemLoader(path or './')) #It means jinja will take templates of environment from file in the introduced path
        
    def sublist(x, y):
        return [x[i] - y[i] if isinstance(x[i], float) else 'NaN' for i in range(len(x))]
    environment.filters['sublist'] = sublist
    def addlist(x, y):
        return [x[i] + y[i] if isinstance(x[i], float) else 'NaN' for i in range(len(x))]
    environment.filters['addlist'] = addlist

    return environment.get_template(filename).render(context)

def add_to_plots(plots, key, x_axis, y_axis):
    for k, plot in plots.items():
        if k == key:
           # plot['data'].append(x_axis)
            plot['data'].append(y_axis)
            plot['x'].append(x_axis)
       # else:
        #    plot['data'].append('NaN')
         #   plot['x'].append('NaN')

    
def gen_plots(filename):
    data = None
    data = copy.deepcopy(pd.read_csv(filename)) 
    #print(data)

    labels = []
    datasets = {'cbands': {'label': ''},
                'electrons': {'label': ''},
               }
    print(datasets.keys())
    print(data)

    #info = []
    #descriptions = set()
    for column in data.columns:
        print(data[column])
        labels.append(column)
        #print(labels)
    for k in list(datasets.keys()):
        if k in data.columns:
                # create data if it does not exist
            if not datasets[k].get('plots', False):
                datasets[k]['plots'] = {}
                #for i, init_desc in enumerate(data['Nodes']):
                datasets[k]['plots']['disc']  = {'x':[], 'data':[], 'color': 'rgba({}, {}, {}, 1)'.format(*colors[1]) }

                # Add to reference or test
            add_to_plots(datasets[k]['plots'], k, data[k], data[k])

            datasets[k]['min'] = min(datasets[k].get('min', 1e12), data[k])
            datasets[k]['max'] = max(datasets[k].get('max', -1e12),data[k])
        else:
            datasets.pop(k)
            continue
                
       # print(datasets[k]['plots'])

            if not datasets[k]['label']:
                try:
                    datasets[k]['label'] = k+' ('+data[k+'_units']+')'
                except KeyError:
                    print("Warning, no units for "+k)

    # Enlarge axis by 5%
#     for k in datasets.keys():
       # variation = (datasets[k]['max'] - datasets[k]['min'])
        # chartjs bug
       # variation = 1e-5 if variation < 1e-5 else variation*0.05

        #datasets[k]['min'] = datasets[k]['min'] - variation
        #datasets[k]['max'] = datasets[k]['max'] + variation

    
    _, basename = os.path.split(filename)
    name, _ = os.path.splitext(basename)
    with open (name+'.html', 'w') as f:
        f.write(render('plot.tmpl', {'title': name, 'labels': labels, 'datasets': datasets}))
    
    
def gen_index(flist):
    entries = {}
    #n=0
    for filename in flist:
        _, basename = os.path.split(filename)
        name, _ = os.path.splitext(basename)
        entries[name+'.html'] = name
        #entries[name+f'{n}'+'.html'] = name+f'{n}'
        #n+=1
    with open ('index.html', 'w') as f:
        f.write(render('index.tmpl', {'entries': entries}))
    #print(entries)

if __name__ == "__main__":
    flist = []
    dataframes_list = [
    {'file': '/home/mandanas/1-CINECA-projects/benchmark/benchmark_10it/bench_7.2dev_gpua_iter/000000/result/result.dat', 'column_name': 'electrons', 'efficiency':'electrons', 'x_axis':'Nodes', 'time_unit':'second'},
    #{'file': '/home/mandanas/1-CINECA-projects/benchmark/benchmark_10it/bench_7.2dev_nogpua_iter/000000/result/result.dat', 'column_name': 'electrons', 'efficiency':'electrons', 'x_axis':'Nodes','time_unit':'second'},
    #{'filename': 'results.dat', 'column_name': 'sth_kernel', 'efficiency':'sth_kernel', 'x_axis':'Nodes', 'time_unit':'second'}
    ]
    filenames = [dataframes_list[i]['file'] for i in range(len(dataframes_list))]
    #print(filenames)
    for filename in filenames:
        gen_plots(filename)
        flist.append(filename)

    gen_index(flist)

dict_keys(['cbands', 'electrons'])
   Nodes  Tasks/Node  Threads/Task  electrons  cbands  cdiaghg  vloc_psi  \
0      1           4             8      53.25   42.58    10.13     10.86   
1      2           4             8      50.65   41.54    10.60     15.86   
2      4           4             8      46.72   38.81    11.12     15.42   

   h_psi  walltime  
0  15.30     60.02  
1  20.46     56.57  
2  20.02     55.84  
0    1
1    2
2    4
Name: Nodes, dtype: int64
0    4
1    4
2    4
Name: Tasks/Node, dtype: int64
0    8
1    8
2    8
Name: Threads/Task, dtype: int64
0    53.25
1    50.65
2    46.72
Name: electrons, dtype: float64
0    42.58
1    41.54
2    38.81
Name: cbands, dtype: float64
0    10.13
1    10.60
2    11.12
Name: cdiaghg, dtype: float64
0    10.86
1    15.86
2    15.42
Name: vloc_psi, dtype: float64
0    15.30
1    20.46
2    20.02
Name: h_psi, dtype: float64
0    60.02
1    56.57
2    55.84
Name: walltime, dtype: float64


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().